# [Neural Networking] Alphabet Soup Charity Funding Predictor (Attempt #1)
## Step #0 - Import Dependencies & Charity CSV Dataset

In [1]:
# Import the relevant modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Step 1: Preprocess the Data
---
### Question #1 - What variable(s) are the target(s) for your model?

The goal is to have the binary classifier predict whether applicants will be successful if funded by Alphabet Soup.

Target Variable = `IS_SUCCESSFUL`

### Question #2 - What variable(s) are the feature(s) for your model?

All of the following variables are features for the model:

- `APPLICATION_TYPE`
- `AFFILIATION`
- `CLASSIFICATION`
- `USE_CASE`
- `ORGANIZATION`
- `STATUS`
- `INCOME_AMT`
- `SPECIAL_CONSIDERATIONS`
- `ASK_AMT`

### Drop the `EIN` and `NAME` columns.

In [2]:
drop_columns = ["EIN", "NAME"]
application_df = application_df.drop(columns = drop_columns)

### Determine the number of unique values for each column.

In [3]:
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


### `APPLICATION_TYPE` - Find the Count for Each Unique Value in the Column and Bin 'Rare' Categorical Variables Together as `Other`

In [4]:
# Log the count of each unique value in the column
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Define a new list to store all the 'rare' unique values in the column
rare_application_types = ["T9", "T13", "T12", "T2",
                          "T25", "T14", "T29", "T15", "T17"]

# Replace all the 'rare' unique values with 'Other' in the column
application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(rare_application_types, "Other")

# Review the counts for each unique value after the replacement
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

### `CLASSIFICATION` - Find the Count for Each Unique Value in the Column and Bin 'Rare' Categorical Variables Together as `Other`

In [6]:
# Save the value_counts series in a new variable; to be used with identifying & replacing the rare unique values
unique_classifications_count = application_df["CLASSIFICATION"].value_counts()

# Log the count of each unique value in the column
unique_classifications_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# Store the threshold count value for 'rare' unique values in the column
rare_classification_threshold_count = 1883

# Return the unique values (indexes in the Series) that have a count less than the pre-defined threshold
# Any unique value with a count less than the threshold is considered 'rare'
rare_classifications = unique_classifications_count[unique_classifications_count < rare_classification_threshold_count].index

# Replace all the 'rare' unique values with 'Other' in the column
application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(rare_classifications, "Other")

# Review the counts for each unique value after the replacement
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

### `pd.get_dummies()` - Encode the Categorical Variables (`numeric_df`) 

In [8]:
# Convert the categorical data to numeric
numeric_df = pd.get_dummies(application_df)

# Columns converted Boolean values can still be considered as numeric i.e. True(1) or False(0)
numeric_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


### Split Pre-Processed Data into Features (`X`) & Target (`y`) DataFrames

In [9]:
# Target Array (y)
y = numeric_df["IS_SUCCESSFUL"]

# Features Array (X)
X = numeric_df.drop(columns = "IS_SUCCESSFUL")

In [10]:
# Preview the Target Array
print(y.value_counts())
print()
y

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64



0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [11]:
# Preview the Features Array
X

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


### Use the `train_test_split` Function to Split the Data into Training and Testing Datasets.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state = 42, 
                                                    stratify = y)

### `StandardScaler` Instance - Fit to the Training Data, then Scale Both Training & Testing Datasets

In [13]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler using the Features Training Dataset
X_scaler = scaler.fit(X_train)

# Scale both the Training & Testing Features Datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Step 2: Compile, Train, and Evaluate the Model
---
### Create the Neural Network Model Using TensorFlow & Keras (`nn_classifier`)

In [14]:
# Get the number of features from the Scaled Training Dataset
input_features_count = len(X_train_scaled[0])

# Pre-define the number of neurons for each hidden layer in the classifier model
nodes_hidden_layer1 = 250
nodes_hidden_layer2 = 150
nodes_hidden_layer3 = 30
nodes_hidden_layer4 = 30

 # Define the deep learning model 
nn_classifier = tf.keras.models.Sequential()

# Applying Hidden (Input) Layer #1
nn_classifier.add(tf.keras.layers.Dense(units = nodes_hidden_layer1, activation = "relu", input_dim = input_features_count))

# Applying Hidden Layer #2
nn_classifier.add(tf.keras.layers.Dense(units = nodes_hidden_layer2, activation = "relu"))

# Applying dropout layer after the 2nd hidden layer
# As the model is being trained, 10% of the neurons' outputs from the 2nd hidden layer are randomly dropped; helps to mitigate overfitting
nn_classifier.add(tf.keras.layers.Dropout(0.1))

# Applying Hidden Layer #3
nn_classifier.add(tf.keras.layers.Dense(units = nodes_hidden_layer3, activation = "relu"))

# Applying Hidden Layer #4
nn_classifier.add(tf.keras.layers.Dense(units = nodes_hidden_layer4, activation = "relu"))

# Applying the Output layer
nn_classifier.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))


# Review the Neural Network Model Classifier Structure
nn_classifier.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 250)               11000     
                                                                 
 dense_1 (Dense)             (None, 150)               37650     
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense_2 (Dense)             (None, 30)                4530      
                                                                 
 dense_3 (Dense)             (None, 30)                930       
                                                                 
 dense_4 (Dense)             (None, 1)                 31        
                                                                 
Total params: 54141 (211.49 KB)
Trainable params: 54141

### Compile & Train the Model (`fit_nn_classifier`)

In [15]:
# Compile the Sequential model together and customise metrics
nn_classifier.compile(loss="binary_crossentropy", optimizer ="adam", metrics=["accuracy"])

# Train the model
fit_nn_classifier = nn_classifier.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100


804/804 [==============================] - 3s 2ms/step - loss: 0.5695 - accuracy: 0.7254
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7308
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7341
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7342
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accu

### Evaluate the Model Using the Test Datasets (`model_loss` / `model_accuracy`)

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_classifier.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5792 - accuracy: 0.7234 - 628ms/epoch - 2ms/step
Loss: 0.5791993737220764, Accuracy: 0.7233819365501404


### Save and Export Results to an HDF5 File (`AlphabetSoupCharity.h5`)

In [17]:
nn_classifier.save("AlphabetSoupCharity.h5")

print("Export to HDF5 File Complete!")

Export to HDF5 File Complete!


c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
